In [ ]:
!git clone https://github.com/UKPLab/argument-reasoning-comprehension-task.git

Cloning into 'argument-reasoning-comprehension-task'...
remote: Enumerating objects: 2593, done.
remote: Total 2593 (delta 0), reused 0 (delta 0), pack-reused 2593 (from 1)
Receiving objects: 100% (2593/2593), 86.00 MiB | 12.71 MiB/s, done.
Resolving deltas: 100% (277/277), done.
Updating files: 100% (2361/2361), done.


In [ ]:
# Specify the path to your file
file_path = "/content/argument-reasoning-comprehension-task/mturk/annotation-task/data/exported-SemEval2018-train-dev-test/test-full.txt"

# Open the file and read its contents as a single string
with open(file_path, "r", encoding="utf-8") as file:
    file_content = file.read()

# Print the content or work with it as a single string
file_content[:1000]  # Display the first 1000 characters for preview to avoid too much output



'#id\twarrant0\twarrant1\tcorrectLabelW0orW1\treason\tclaim\tdebateTitle\tdebateInfo\n18249360_112_A104V8NZIQFN2F\tcomments sections are a welcome distraction from my work\tcomments sections always distract me from my work\t0\tThey add a lot to the piece and I look forward to reading comments.\tComment sections have not failed\tHave Comment Sections Failed?\tIn recent years, many media companies have disabled them because of widespread abuse and obscenity.\n18248741_114_A104V8NZIQFN2F\tthe rest of the comments can be skipped easily\tthe rest of the comments make the section unreadable\t0\tMany comments are thoughful and insightful.\tComment sections have not failed\tHave Comment Sections Failed?\tIn recent years, many media companies have disabled them because of widespread abuse and obscenity.\n18362833_247_AE861G0AY5RGT\tall other search engines re-direct to google\tother search engines do not re-direct to google\t1\tPeople can choose not to use Google.\tGoogle is not a harmful monop

In [ ]:
# Re-attempting the parsing process without including the correct label.
# The focus will be on reason, claim, warrant0, and warrant1 for each ID.

# Dictionary to hold parsed data by ID
parsed_data = {}
correct_labels = {}

try:
    with open(file_path, 'r', encoding='utf-8') as file:
        # Skip the header line
        next(file)

        # Read and parse each line
        for line in file:
            # Split the line by tab
            parts = line.strip().split('\t')

            # Ensure the line has the expected number of columns
            if len(parts) >= 8:
                # Assign values to each variable
                instance_id = parts[0]
                warrant0 = parts[1]
                warrant1 = parts[2]
                reason = parts[4]
                claim = parts[5]
                debate_title = parts[6]
                debate_info = parts[7]

                # Store data in a structured dictionary by instance ID, excluding the correct label
                parsed_data[instance_id] = {
                    "reason": reason,
                    "claim": claim,
                    "warrant0": warrant0,
                    "warrant1": warrant1,
                    "debate_title": debate_title,
                    "debate_info": debate_info
                }
                correct_labels[instance_id] = parts[3]

except FileNotFoundError as e:
    parsed_data = str(e)

# Display a sample of the parsed data (first 5 instances)



In [ ]:
list(parsed_data.items())[:5]

[('18249360_112_A104V8NZIQFN2F',
  {'reason': 'They add a lot to the piece and I look forward to reading comments.',
   'claim': 'Comment sections have not failed',
   'warrant0': 'comments sections are a welcome distraction from my work',
   'warrant1': 'comments sections always distract me from my work',
   'debate_title': 'Have Comment Sections Failed?',
   'debate_info': 'In recent years, many media companies have disabled them because of widespread abuse and obscenity.'}),
 ('18248741_114_A104V8NZIQFN2F',
  {'reason': 'Many comments are thoughful and insightful.',
   'claim': 'Comment sections have not failed',
   'warrant0': 'the rest of the comments can be skipped easily',
   'warrant1': 'the rest of the comments make the section unreadable',
   'debate_title': 'Have Comment Sections Failed?',
   'debate_info': 'In recent years, many media companies have disabled them because of widespread abuse and obscenity.'}),
 ('18362833_247_AE861G0AY5RGT',
  {'reason': 'People can choose n

In [ ]:
list(correct_labels.items())[:5]

[('18249360_112_A104V8NZIQFN2F', '0'),
 ('18248741_114_A104V8NZIQFN2F', '0'),
 ('18362833_247_AE861G0AY5RGT', '1'),
 ('19120938_547_A1I4CYG5YDFTYM', '0'),
 ('18541016_156_A1DJNUJZN8FE7N', '1')]

In [ ]:
len(parsed_data)

444

TESTING WITH LLAMA

In [ ]:

!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your termin

In [ ]:
import transformers
import torch
import os
HF_TOKEN = os.getenv("HF_TOKEN")
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 96.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not cu

TESTING WITH SINGLE TEST QN

In [ ]:
# Define the example question data in the desired format
example_instance_id = list(parsed_data.keys())[0]
data = parsed_data[example_instance_id]

# Prepare the prompt in the message format
messages = [
    {
        "role": "system",
        "content": "You are an assistant who provides answers to logical reasoning questions. Use Chain of Thought reasoning but don't give the Chain of Thought output, just give the single digit correct warrant option (0 or 1)."
    },
    {
        "role": "user",
        "content": f"""
        Reason: {data['reason']}
        Claim: {data['claim']}

        Which of the following warrants makes the claim logically consistent with the Reason?

        Option 0: {data['warrant0']}
        Option 1: {data['warrant1']}

        Please respond with the correct option number (0 or 1).
        """
    }
]


output = pipeline(messages, max_new_tokens=1)

    # Extract the predicted answer
predicted_answer = output[0]["generated_text"][-1]['content']  # This should be either '0' or '1'

    # Retrieve the correct answer
correct_answer = correct_labels[example_instance_id]

    # Check if the model's prediction is correct
is_correct = (predicted_answer == correct_answer)

    # Display the results
result = {
        "instance_id": example_instance_id,
        "prompt": messages[1]["content"],
        "predicted_answer": predicted_answer,
        "correct_answer": correct_answer,
        "is_correct": is_correct
}
print(result)



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'instance_id': '18249360_112_A104V8NZIQFN2F', 'prompt': '\n        Reason: They add a lot to the piece and I look forward to reading comments.\n        Claim: Comment sections have not failed\n\n        Which of the following warrants makes the claim logically consistent with the premise?\n\n        Option 0: comments sections are a welcome distraction from my work\n        Option 1: comments sections always distract me from my work\n\n        Please respond with the correct option number (0 or 1).\n        ', 'predicted_answer': '1', 'correct_answer': '0', 'is_correct': False}


TESTING ON ENTIRE DATASET

In [ ]:
import transformers
import torch

# Set up the Hugging Face pipeline with your model
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

# Initialize counters and storage for results
correct_predictions = 0
total_instances = len(parsed_data)
results = []

# Loop through each instance in parsed_data
for instance_id, data in parsed_data.items():
    # Prepare the prompt in the message format
    messages = [
        {
            "role": "system",
            "content": "You are an assistant who provides answers to logical reasoning questions. Use Chain of Thought reasoning but don't give the Chain of Thought output, just give the single digit correct warrant option (0 or 1)."
        },
        {
            "role": "user",
            "content": f"""
            Reason: {data['reason']}
            Claim: {data['claim']}

            Which of the following warrants makes the claim logically consistent with the premise?

            Option 0: {data['warrant0']}
            Option 1: {data['warrant1']}

            Please respond with the correct option number (0 or 1).
            """
        }
    ]

    # Generate the model's response
    try:
        output = pipeline(messages, max_new_tokens=1)

        # Extract the predicted answer
        predicted_answer = output[0]["generated_text"][-1]['content'] # Should be '0' or '1'

        # Retrieve the correct answer
        correct_answer = correct_labels[instance_id]

        # Check if the model's prediction is correct
        is_correct = (predicted_answer == correct_answer)
        if is_correct:
            correct_predictions += 1

        # Store the result for this instance
        results.append({
            "instance_id": instance_id,
            "predicted_answer": predicted_answer,
            "correct_answer": correct_answer,
            "is_correct": is_correct
        })

        # Print each instance's result
        print(f"Instance ID: {instance_id}, Correct Answer: {correct_answer}, Predicted Answer: {predicted_answer}")

    except Exception as e:
        print(f"Error processing instance {instance_id}: {e}")

# Calculate and print overall accuracy
accuracy = correct_predictions / total_instances if total_instances > 0 else 0
print(f"Total Instances: {total_instances}, Correct Predictions: {correct_predictions}, Accuracy: {accuracy:.2%}")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Instance ID: 18249360_112_A104V8NZIQFN2F, Correct Answer: 0, Predicted Answer: 1


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Instance ID: 18248741_114_A104V8NZIQFN2F, Correct Answer: 0, Predicted Answer: 1


...

Instance ID: 19647276_194_A1DJNUJZN8FE7N, Correct Answer: 1, Predicted Answer: 1


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Instance ID: 18365640_74_APW9F8OTJ4KXO, Correct Answer: 1, Predicted Answer: 1


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Instance ID: 17136757_110_A34QZDSTKZ3JO9, Correct Answer: 0, Predicted Answer: 1
Instance ID: 19585313_0_A104V8NZIQFN2F, Correct Answer: 0, Predicted Answer: 1
Total Instances: 444, Correct Predictions: 241, Accuracy: 54.28%


In [ ]:
# Calculate and print accuracy
accuracy = (correct_predictions / total_instances) * 100
print(f"Accuracy: {accuracy:.2f}%")



Accuracy: 54.28%


In [ ]:
# Display some of the results for verification
print(results[:5])  # Print the first 5 results for inspection

[{'instance_id': '18249360_112_A104V8NZIQFN2F', 'predicted_answer': '1', 'correct_answer': '0', 'is_correct': False}, {'instance_id': '18248741_114_A104V8NZIQFN2F', 'predicted_answer': '1', 'correct_answer': '0', 'is_correct': False}, {'instance_id': '18362833_247_AE861G0AY5RGT', 'predicted_answer': '1', 'correct_answer': '1', 'is_correct': True}, {'instance_id': '19120938_547_A1I4CYG5YDFTYM', 'predicted_answer': '1', 'correct_answer': '0', 'is_correct': False}, {'instance_id': '18541016_156_A1DJNUJZN8FE7N', 'predicted_answer': '1', 'correct_answer': '1', 'is_correct': True}]


In [ ]:
import pandas as pd

# Convert the results to a DataFrame
df_results = pd.DataFrame([
    {
        "instance_id": result["instance_id"],
        "warrant0": parsed_data[result["instance_id"]]['warrant0'],
        "warrant1": parsed_data[result["instance_id"]]['warrant1'],
        "reason": parsed_data[result["instance_id"]]['reason'],
        "claim": parsed_data[result["instance_id"]]['claim'],
        "correctLabelW0orW1": result["correct_answer"],
        "predictedLabelW0orW1": result["predicted_answer"],
        "is_correct": result["is_correct"]
    }
    for result in results
])

# Save the DataFrame to a CSV file
file_path = '/content/prediction.csv'
df_results.to_csv(file_path, index=False)

file_path


'/content/prediction.csv'

TESTING WITH GEMINI PRO

In [ ]:
# Import the SDK
import google.generativeai as genai

# Set the API key directly
GOOGLE_API_KEY = "api-key"  # Replace with your actual API key
genai.configure(api_key=GOOGLE_API_KEY)


In [ ]:
model = genai.GenerativeModel('gemini-1.5-pro')

In [ ]:
if parsed_data:  # Check if parsed_data is not empty
    first_instance_id = list(parsed_data.keys())[0]
    first_instance_data = parsed_data[first_instance_id]
    correct_answer = correct_labels[first_instance_id]  # Retrieve the correct label

    # Generate the prompt for the first question
    prompt = f"""
    Premise: {first_instance_data['reason']}
    Claim: {first_instance_data['claim']}

    Which of the following warrants makes the claim logically consistent with the premise?

    Option 0: {first_instance_data['warrant0']}
    Option 1: {first_instance_data['warrant1']}

    Please respond with the correct option number (0 or 1).
    """

    # Generate the response from the model
    try:
        response = model.generate_content(
            prompt,
            generation_config=genai.types.GenerationConfig(max_output_tokens=1, temperature=0)
        )

        # Extract and clean the predicted answer
        predicted_answer = response.text.strip()  # Ensure it's a clean "0" or "1"

        # Compare predicted answer with correct answer
        is_correct = predicted_answer == correct_answer
        print(f"Predicted Answer: {predicted_answer}")
        print(f"Correct Answer: {correct_answer}")
        print(f"Is the prediction correct? {'Yes' if is_correct else 'No'}")

    except Exception as e:
        print(f"Error generating response: {e}")

Predicted Answer: 0
Correct Answer: 0
Is the prediction correct? Yes


In [ ]:
import google.generativeai as genai
import time

# Initialize counters
correct_predictions = 0
total_instances = len(parsed_data)

# Loop through each instance in the dataset
for instance_id, data in parsed_data.items():
    time.sleep(15)  # Adjust delay based on your quota limits
    # Generate the prompt for each question
    prompt = f"""
    Premise: {data['reason']}
    Claim: {data['claim']}

    Which of the following warrants makes the claim logically consistent with the premise?

    Option 0: {data['warrant0']}
    Option 1: {data['warrant1']}

    Please respond with the correct option number (0 or 1).
    """

    # Generate the response from the model

    response = model.generate_content(
            prompt,
            generation_config=genai.types.GenerationConfig(max_output_tokens=1)
        )

        # Extract and clean the predicted answer
    predicted_answer = response.text.strip()

        # Retrieve the correct answer from correct_labels
    correct_answer = correct_labels[instance_id]

        # Check if the prediction is correct
    if predicted_answer == correct_answer:
        correct_predictions += 1

        # Print each prediction result (optional for monitoring)
    print(f"Instance ID: {instance_id}, Predicted Answer: {predicted_answer}, Correct Answer: {correct_answer}")






In [ ]:
# Calculate accuracy
accuracy = (correct_predictions / total_instances) * 100
print(f"Accuracy: {accuracy:.2f}%")
